<a href="https://colab.research.google.com/github/Rusira54321/RAG-APPLICATION-Medium-level-/blob/main/ragapplication1234.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#install the necessary packages
!pip install langchain -qU
!pip install langchain-openai -qU
!pip install langchain-chroma -qU
!pip install langchain_community -qU

In [3]:
import os
from google.colab import userdata

In [4]:
from langchain_openai import ChatOpenAI
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_KEY')
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)

In [5]:
from langchain_openai import OpenAIEmbeddings
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [6]:
!pip install pypdf -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.6/329.6 kB 29.7 MB/s eta 0:00:00


In [7]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/codeprolk.pdf")
docs = loader.load()
print(docs)

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2024-06-18T10:00:42+05:30', 'author': 'Dinesh Piyasamara', 'moddate': '2024-06-18T10:00:42+05:30', 'source': '/content/codeprolk.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content="Introduction to CodePRO LK \nCodePRO LK is a dynamic educational platform that offers a diverse range of technology-\nrelated courses in Sinhala, aimed at empowering Sri Lankans with valuable skills in \nprogramming, data science, and machine learning. Founded by Dinesh Piyasamara during the \nCOVID-19 pandemic, CodePRO LK addresses the growing need for accessible, high-quality \ntech education tailored to the local community. \n \nFounding and Vision \nOrigin and Motivation \nThe inception of CodePRO LK was driven by the challenges posed by the COVID-19 pandemic, \nwhich highlighted the necessity for remote learning and digital skills. Recognizing this, Dines

In [11]:
len(docs)

4

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter  = RecursiveCharacterTextSplitter(chunk_size=400,chunk_overlap=50)
splites = text_splitter.split_documents(docs)

In [13]:
len(splites)

20

In [14]:
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(documents=splites,embedding=embedding_model)

In [15]:
retriever = vectorstore.as_retriever()

In [35]:
from langchain_core.prompts import ChatPromptTemplate
system_prompt = (
     "You are an intelligent chatbot. Use the following context to answer the question. If you don't know the answer, just say that you don't know."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{query}"),
    ]
)

In [36]:
prompt

ChatPromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an intelligent chatbot. Use the following context to answer the question. If you don't know the answer, just say that you don't know.\n\n{context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})])

In [41]:
from langchain_classic.chains.retrieval_qa.base import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt":prompt},
    return_source_documents=False,
    input_key="query"
)

In [44]:
response = qa_chain({"query": "Who is codeprolk"})
print(response)

/tmp/ipython-input-1967582856.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  response = qa_chain({"query": "Who is codeprolk"})


ValueError: Missing some input keys: {'query'}